In [17]:
from __future__ import print_function

import lda
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.manifold import TSNE
from pymongo import MongoClient
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 100)
import nltk
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import re
import string
import sys
import os

import jellyfish # library for fuzzy text comparisons
from sklearn.model_selection import train_test_split
from scipy.spatial.distance import cdist
import matplotlib.pyplot as plt

from tqdm import tqdm_pandas, tqdm_notebook as tqdm
from tqdm import tqdm as tqorig
tqorig.pandas(tqdm)
from autocorrect import spell
from textblob import TextBlob

[nltk_data] Downloading package wordnet to /Users/natali/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [14]:
import sys
if sys.version_info[0] >= 3:
    unicode = str

In [2]:
client = MongoClient()
db     = client.fletcher
reviews   = db.reviews

In [4]:
cursor = reviews.find({},{ 'text_review Student Support': 1, 'college' : 1, 'text_review Job Prospects': 1, 'text_review Clubs and Societies':1,
                        } )

cursor = reviews.find({},{ 'text_review Student Support': 1, 'college' : 1, 'text_review Job Prospects': 1, 'text_review Clubs and Societies':1,
                         'text_review City Life': 1,'text_review Uni Facilities': 1, "text_review Student Support" : 1} )

In [5]:
df_overall_review = pd.DataFrame(list(cursor))

In [6]:
#df_overall_review['reviews'] = df_overall_review['text_review OVERALL UNIVERSITY RATING']
df_overall_review = df_overall_review.dropna()

In [7]:
df_overall_review.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 42042 entries, 0 to 86428
Data columns (total 5 columns):
_id                                42042 non-null object
college                            42042 non-null object
text_review Clubs and Societies    42042 non-null object
text_review Job Prospects          42042 non-null object
text_review Student Support        42042 non-null object
dtypes: object(5)
memory usage: 1.9+ MB


In [8]:
letters_only = lambda x: re.sub("[^a-zA-Z]", " ", str(x))
alphanumeric = lambda x: re.sub('\w*\d\w*', ' ', x)
punc_lower = lambda x: re.sub('[%s]' % re.escape(string.punctuation), ' ', x.lower())

In [19]:
df_overall_review['text_review Student Support'] = df_overall_review['text_review Student Support'].map(letters_only).map(alphanumeric).map(punc_lower)

In [20]:
df_overall_review['text_review Job Prospects'] = df_overall_review['text_review Job Prospects'].map(letters_only).map(alphanumeric).map(punc_lower)

In [24]:
df_overall_review['text_review Clubs and Societies'] = df_overall_review['text_review Clubs and Societies'].map(letters_only).map(alphanumeric).map(punc_lower)

In [22]:
text = list(df_overall_review['text_review Student Support']) + list(df_overall_review['text_review Job Prospects']) + list(df_overall_review['text_review Clubs and Societies']) 

In [438]:
n_topics = 3 # number of topics
n_iter = 500 # number of iterations

In [40]:
from sklearn.manifold import TSNE

In [50]:
def lda_pic(university, df):
    df[df['college'] == university][0:400]
    text = [item for item in df]
    news = [' '.join(filter(unicode.isalpha, raw.lower().split())) for raw in text]
    cvectorizer = CountVectorizer(min_df=2, stop_words='english')
    cvz = cvectorizer.fit_transform(news)

    
    n_topics = 3 # number of topics
    n_iter = 500 # number of iterations
    # train an LDA model
    lda_model = lda.LDA(n_topics=n_topics, n_iter=n_iter)
    X_topics = lda_model.fit_transform(cvz)
    
    n_top_words = 3
    colormap = np.array(["#1f77b4", "#aec7e8", "#ff7f0e"])
    _lda_keys = []
    for i in range(X_topics.shape[0]):
        _lda_keys += X_topics[i].argmax(),
    topic_summaries = []
    topic_word = lda_model.topic_word_  # all topic words
    vocab = cvectorizer.get_feature_names()
    for i, topic_dist in enumerate(topic_word):
        topic_words = np.array(vocab)[np.argsort(topic_dist)][:-(n_top_words + 1):-1] # get!
        topic_summaries.append(' '.join(topic_words)) # a
    
    tsne_model = TSNE(n_components=2, verbose=1, random_state=0, angle=.99, init='pca')
    tsne_lda = tsne_model.fit_transform(X_topics)
    
    plt.scatter(x=tsne_lda[:, 0], y=tsne_lda[:, 1],
                 color=colormap[_lda_keys])

In [51]:
lda_pic(university = 'University of Hull', df = df_overall_review)

ValueError: After pruning, no terms remain. Try a lower min_df or a higher max_df.

In [45]:
plt.scatter(x=tsne_lda[:, 0], y=tsne_lda[:, 1],
                 color=colormap[_lda_keys][:num_example])

NameError: name 'tsne_lda' is not defined

In [472]:
first_class_label = _lda_keys[:201]   #topic 0
zero_class_label = _lda_keys[201:401]
second_class_label = _lda_keys[401:601]

In [ ]:
def accuracy(_lda_keys):
    
    count = 0
    for item in _lda_keys[0:201]:
        if item == 1:
            count += 1


In [482]:
count = 0
for item in first_class_label:
    if item == 1:
        count += 1

In [491]:
count = 0
for item in second_class_label:
    if item == 2:
        count += 1

In [492]:
count/200

0.745

In [471]:
first_class_label[[first_class_label] == 1]

1

In [452]:
topic_coord

array([[-11.29643631,   1.73261952],
       [ -5.78327465,  12.13912201],
       [ -0.31631407,  22.53479576],
       [ 11.08636951,  -9.64781189],
       [  0.93835849, -22.56160164],
       [  1.69799638,  -3.31839275],
       [  8.30171013,   4.65296078],
       [-16.74311829,  -5.95915604],
       [ 18.37014008, -14.83756733],
       [-21.69774246,   6.16550732],
       [ -2.39691424,  -9.39934731],
       [ -4.7081995 ,  -4.71832418],
       [  0.28665635,   2.00243735],
       [ -6.49097538, -14.55294132],
       [ 17.20210266,  16.30933762],
       [-13.11538982,  28.14277458],
       [  6.03042364,  13.00139141],
       [  3.56107664,   3.5715704 ],
       [ 21.2578392 ,   2.85401559],
       [  3.42163467, -21.77051926]])

In [453]:
topic_summaries[0]

'skills careers work'

In [454]:
topic_summaries[1]

'support good academic'

In [455]:
topic_summaries[2]

'societies clubs good'

In [73]:
topic_coord = np.empty((X_topics.shape[1], 2)) * np.nan
for topic_num in _lda_keys:
    if not np.isnan(topic_coord).any():
        break
    topic_coord[topic_num] = tsne_lda[_lda_keys.index(topic_num)]

# plot crucial words
for i in range(X_topics.shape[1]):
    plt.text(topic_coord[i, 0], topic_coord[i, 1], [topic_summaries[i]])